In [1]:
# --- Imports ---
import pandas as pd
import time
import ast 
from openai import OpenAI

# --- Set up OpenAI client (replace with your key) ---
client = OpenAI(api_key="")

# --- Load and prepare variable names ---
df = pd.read_excel("Dictionary_cleaned.xlsx")
variable_list = df["Variable"].dropna().tolist()

# --- Build the prompt for a given block of variable names ---
def build_prompt(variable_names, block_num):
    """
    Construct a prompt instructing the model to categorize a list of variable names.

    Args:
        variable_names (List[str]): List of variable names to categorize.
        block_num (int): Sequential block number for reference.

    Returns:
        str: Formatted prompt for GPT model.
    """
    variable_text = "\n".join([f"{i+1}. {v}" for i, v in enumerate(variable_names)])
    prompt = f"""
You are reviewing a list of variable names from a pregnancy and child development research dataset.

Please analyze the following variables and suggest 5 to 10 distinct, non-overlapping semantic categories that could be used to group them.

The categories must be:
- concise (2–5 words each)
- non-redundant
- general enough to group similar variables, but specific enough to be meaningful
- specify clearly whether the category relates to the **mother**, the **child**, or **both**

Return your result as a valid Python list of strings.

Block #{block_num}
Here is the list of variable names:
{variable_text}
"""
    return prompt.strip()

In [2]:
# --- Generate prompts in blocks and collect model responses ---
block_size = 100
results = []

for i in range(0, len(variable_list), block_size):
    block = variable_list[i:i + block_size]
    block_num = i // block_size + 1
    prompt = build_prompt(block, block_num)

    try:
        response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[
                {"role": "system", "content": "You are a data categorization expert."},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
        categories = response.choices[0].message.content.strip()
        results.append({
            "block": block_num,
            "prompt_sample": block[:3],  # Save a preview of the input
            "categories": categories     # Raw model output
        })

        print(f"Block {block_num} completed successfully.")
        time.sleep(1)  # Add delay to respect API rate limits

    except Exception as e:
        print(f"Block {block_num} failed with error: {e}")
        results.append({
            "block": block_num,
            "prompt_sample": block[:3],
            "categories": f"ERROR: {e}"
        })

Block 1 completed successfully.
Block 2 completed successfully.
Block 3 completed successfully.
Block 4 completed successfully.
Block 5 completed successfully.
Block 6 completed successfully.
Block 7 completed successfully.
Block 8 completed successfully.
Block 9 completed successfully.
Block 10 completed successfully.
Block 11 completed successfully.
Block 12 completed successfully.
Block 13 completed successfully.
Block 14 completed successfully.
Block 15 completed successfully.
Block 16 completed successfully.


In [3]:
# --- Extract and parse model-generated categories from each block ---
all_categories = []

for response_n in range(len(results)):
    raw = results[response_n]['categories']
    
    if raw.startswith("ERROR:"):
        # Skip blocks that failed during generation
        continue

    # Clean output formatting (remove Markdown/code block formatting)
    cleaned = raw.strip().removeprefix("```python").removesuffix("```").strip()

    try:
        parsed_list = ast.literal_eval(cleaned)  # Convert string list to Python list
        all_categories += parsed_list
    except (ValueError, SyntaxError) as e:
        print(f"Failed to parse block {response_n + 1}: {e}")
        continue

# --- Deduplicate and display results ---
unique_categories = list(set(all_categories))
print(f"Total unique categories: {len(unique_categories)}")

Total unique categories: 72


In [11]:
unique_categories

['Child MRI Data',
 'Child Age and Demographics',
 'Child Developmental Summary Scores',
 'Child Physical Measurements',
 'Child Social-Emotional Skills',
 'Mother-Child Environment',
 'Child Problem-Solving Skills',
 'Child Developmental Screening',
 'Parental Perceptions and Behaviors',
 'Child Identification Codes',
 'Mother Mental Health',
 'Childcare and Living Arrangements',
 'Mother Medications',
 'Child Feeding Practices',
 'Child Twin Status',
 'Mother Demographics',
 'Child Feeding and Breastfeeding',
 'Child Anthropometrics',
 'Child Temperament',
 'Study Administration (Both)',
 'Child Cognitive Assessments',
 'Mother Household Context',
 'Mother Physical Health',
 'Mother Feeding Practices',
 'Infant Feeding Practices',
 'Administrative Identifiers',
 'Child Physical Growth',
 'Mother Pregnancy History',
 'Mother-Partner Relationship',
 'Maternal Life Events',
 'Mother Socioeconomic Status',
 'Child Demographics',
 'Child Study Metadata',
 'Parental Perceptions/Behaviors (

In [6]:
import os
import ast
from openai import OpenAI

# --- Initialize OpenAI client ---
client = OpenAI(api_key="")

# --- Input: list of categories to consolidate ---
categories = unique_categories  # Assumes this variable is already defined

# --- Prompt builder ---
def build_mapping_prompt(category_list):
    cat_text = "\n".join([f"{i+1}. {cat}" for i, cat in enumerate(category_list)])
    return f"""
You are given a list of ~70 overlapping variable categories from a pregnancy and child development dataset.

Your task is to **group similar or overlapping categories into 30 to 35 consolidated, non-overlapping categories**, and return a **Python dictionary** where:

- Each key is one of the original input categories.
- Each value is the name of the consolidated category it belongs to.
- The output must be a valid Python dictionary with string keys and string values.
- Do not explain or format — only return the dict.

Here is the full list:
{cat_text}
""".strip()

# --- Send request and parse response as Python dict ---
prompt = build_mapping_prompt(categories)
response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {"role": "system", "content": "You are a data categorization expert."},
        {"role": "user", "content": prompt}
    ],
    temperature=0
)

raw_text = response.choices[0].message.content.strip()

try:
    cleaned = raw_text.removeprefix("```python").removesuffix("```").strip()
    mapping_dict = ast.literal_eval(cleaned)

    # Get list of consolidated category values
    consolidated_list = list(set(mapping_dict.values()))
    print(len(consolidated_list))

except (SyntaxError, ValueError) as e:
    print("Error parsing model response as dictionary:", e)
    print("Raw response:")
    print(raw_text)

33


In [12]:
consolidated_list

['Child Birth and Twin Status',
 'Mother-Child Environment',
 'Study Administration and Metadata',
 'Parental Perceptions and Behaviors',
 'Child Motor Skills',
 'Mother Mental Health',
 'Childcare and Living Arrangements',
 'Mother Demographics',
 'Child Cognitive and Problem-Solving Skills',
 'Mother Pregnancy and Delivery History',
 'Mother Sleep Patterns',
 'Mother Household Context',
 'Mother Feeding Practices',
 'Administrative Identifiers',
 'Mother-Partner Relationship',
 'Maternal Life Events',
 'Mother Socioeconomic Status',
 'Child Demographics',
 'Child Temperament and Behavior',
 'Child Developmental Assessments',
 'Child Language and Communication',
 'Child MRI and Neuroimaging',
 'Child Social-Emotional Development',
 'Child Feeding and Nutrition',
 'Child Sleep Patterns',
 'Child General Health',
 'Mother Employment',
 'Mother Substance Use',
 'Child Screen Time',
 'Mother Social Support',
 'COVID-19 Status and Impact',
 'Child Anthropometrics and Physical Growth',
 'Mo

In [8]:
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import os

# Load cleaned variable definitions
df = pd.read_excel('Dictionary_cleaned.xlsx')
df = df[df['Label'].notna()].reset_index(drop=True)
df_subset = df[['Variable', 'Label']].copy()

In [9]:
# Initialize OpenAI client
client = OpenAI(api_key="")

# Build system prompt using dynamic category list
system_prompt = (
    "You are a data classification assistant working with a maternal and child development dataset. "
    "You will classify each variable description into one of the predefined categories.\n\n"
    "The categories are:\n"
    + "\n".join(f"- {cat}" for cat in categories)
    + "\n\nReturn only the exact name of the category for each variable, in the original order."
)

def classify_batch(batch_df):
    """
    Classify a batch of variables into predefined GPT-generated categories.
    """
    prompt = "Here are the variables:\n\n"
    for i, row in batch_df.iterrows():
        prompt += f"{i+1}. {row['Label']}\n\n"

    prompt += "Return the list of categories, one per variable, matching the numbering."

    try:
        response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

# Run batch classification
batch_size = 25
for start in tqdm(range(0, len(df_subset), batch_size)):
    end = min(start + batch_size, len(df_subset))
    batch_df = df_subset.iloc[start:end].reset_index(drop=True)
    output = classify_batch(batch_df)

    # Parse and assign model outputs
    lines = output.split('\n')
    clean_categories = []
    for line in lines:
        if '.' in line:
            category = line.split('.', 1)[1].strip()
            clean_categories.append(category)
        else:
            clean_categories.append(line.strip())

    for idx, cat in enumerate(clean_categories):
        df_subset.loc[start + idx, 'GPTCategory'] = cat


100%|██████████████████████████████████████████████████████████████████████████████████| 62/62 [01:40<00:00,  1.62s/it]


In [ ]:
# Save final dataset with GPT-assigned categories
df_subset.to_excel("categorized_variables.xlsx", index=False)

In [10]:
df_subset

,Variable,Label,GPTCategory
0,ID,Participant Study ID,Administrative Identifiers
1,ID2,Numeric Participant Study ID,Administrative Identifiers
2,record_id,Record ID,Administrative Identifiers
3,preg_demo_timespreg,How many TOTAL times have you been pregnant (i...,Mother Pregnancy History
4,preg_demo_duedate,Approximately when is the due date to deliver ...,Pregnancy and Delivery Details
...,...,...,...
1530,nep_sn_tottime_percentrank_age3_4_bb2,Speed Naming Total Completion Time Percentile ...,Child Cognitive Assessments
1531,nep_sn_combined_scaleds_age3_4_bb2,SCALED SCORE - SN Combined score,Child Cognitive Assessments
1532,nep_sn_combinedscalescore_percent_age3_4_bb2,SN Combined Scaled Score Percentile,Child Cognitive Assessments
1533,nep_sn_combined_scaleds_age3_5_bb2,SN Combined SCALED SCORE,Child Cognitive Assessments
